In [288]:
import ipywidgets as widgets

In [289]:
import pandas as pd
import yfinance as yf
import yahoofinancials

In [290]:
import bs4 as bs
import pickle
import requests

In [291]:
import hvplot.pandas

In [292]:
import datetime

In [293]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
#     with open("sp500tickers.pickle","wb") as f:
#         pickle.dump(tickers,f)
        
    return tickers

abbr_list = save_sp500_tickers()
abbr_list = [f.strip() for f in abbr_list]

column_list = ['all', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [315]:
abbr_widget = widgets.Dropdown(options=[(i, i) for i in abbr_list])
manual_abbr = widgets.Text()
column_widget = widgets.Dropdown(options=column_list)

plot_layout = widgets.Layout(display='flex',
#                     flex_flow='space-between',
    justify_content='space-between',
                    width='100%')

plot_out = widgets.Output(layout=plot_layout)

box_layout = widgets.Layout(display='flex',
#                     flex_flow='space-between',
    justify_content='space-between',
                    width='50%')
# box_layout = widgets.Layout(display='flex',
#                     flex_flow='column',
#                     align_items='stretch',
#                     border='solid',
#                     width='50%')

plot_size_widget = widgets.IntSlider(value=700, min=350, max=1500, step=50)
date_start = widgets.DatePicker(value=datetime.datetime.today() - datetime.timedelta(days=30))
date_end = widgets.DatePicker(value=datetime.datetime.today())
submit = widgets.Button(description='Generate Plot', button_style='success')

def manual_abbr_submit(change):
    print(change)
    
# manual_abbr.observe(manual_abbr_submit)
manual_abbr.unobserve_all()

def submit_func(button):
    abbr = abbr_widget.value if len(manual_abbr.value) == 0 else manual_abbr.value#give the manual input precedence
    column = column_widget.value if column_widget.value != 'all' else ['Open', 'High', 'Low', 'Close', 'Adj Close']

    df = yf.download(
        abbr, 
        start=datetime.datetime.strftime(date_start.value, '%Y-%m-%d'), 
        end=datetime.datetime.strftime(date_end.value, '%Y-%m-%d'), 
        progress=False
    )
    with plot_out:
        display(df[column].hvplot(title=abbr, hover_cols='all', width=plot_size_widget.value, height=int((3/7)*plot_size_widget.value)))
    plot_out.clear_output(wait=True)
    
submit.on_click(submit_func)



In [316]:
abbr_widget.unobserve_all()

In [317]:
def viz_func(change):
#     print(change['name'])
    if change['name'] != 'index':
        return
    abbr = abbr_list[change['new']]
    column = column_widget.value if column_widget.value != 'all' else ['Open', 'High', 'Low', 'Close', 'Adj Close']
    df = yf.download(
        abbr, 
        start=datetime.datetime.strftime(date_start.value, '%Y-%m-%d'), 
        end=datetime.datetime.strftime(date_end.value, '%Y-%m-%d'), 
        progress=False
    )
    with plot_out:
#         display(df[column_widget.value].hvplot.line(hover_cols='all', width=1400, height=600))
        display(df[column].hvplot.line(title=abbr, hover_cols='all', width=plot_size_widget.value, height=int((3/7)*plot_size_widget.value)))
    plot_out.clear_output(wait=True)
#     print(change['name'], change.keys())
#     print(change)

In [318]:
# hvplot.help('line')

In [319]:
abbr_widget.observe(viz_func)

In [320]:
widgets.HTML(value='<h1 style="font-family: Default-Monospace; color:navy;">Devan\'s Stock Visualization Application</h1>')

HTML(value='<h1 style="font-family: Default-Monospace; color:navy;">Devan\'s Stock Visualization Application</…

In [321]:
display(widgets.VBox(
    [
        widgets.HBox([widgets.Label('Stock Abbreviation'), abbr_widget], layout=box_layout),
        widgets.HBox([widgets.Label('Unlisted Abbreviation'), manual_abbr], layout=box_layout),
        widgets.HBox([widgets.Label('Data Facet'), column_widget], layout=box_layout),
        widgets.HBox([widgets.Label('Start Date'), date_start], layout=box_layout),
        widgets.HBox([widgets.Label('End Date'), date_end], layout=box_layout),
        widgets.HBox([widgets.Label('Plot Size'), plot_size_widget], layout=box_layout),
        widgets.HBox([widgets.Label(''), submit], layout=box_layout),
        plot_out,
    ]
), box_layout)

Layout(display='flex', justify_content='space-between', width='50%')